# Домашнее задание № 8

## Задание 1 (4 балла)

Обучите 8 моделей для задачи классификации текста (датасет - lenta_40k ). А именно:  
1) модель с 1 GRU слоем;   
2) модель с 1 LSTM слоем    
3) модель с 1 GRU и 1 LSTM слоем  
4) модель с 1 BIGRU и 2 LSTM слоями  
7) модель с 7 GRU слоями и 3 LSTM слоями  
6) модель 1 BIGRU и 1 BILSTM слоями, причем так чтобы модели для forward и backward прохода отличались   
7) модель, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM  



Параметр units и размер эмбединга можете задать любой. Оцените качество каждой модели и определите победителя.

In [1]:
!pip install pandas scikit-learn matplotlib tensorflow

In [2]:
import tensorflow as tf

2024-02-22 11:34:46.794585: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 11:34:46.794676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 11:34:46.799235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 11:34:46.813412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 11:34:47.834629: W tensorflow/compiler/tf2

In [3]:
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML
import matplotlib.pyplot as plt

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [5]:
data = pd.read_csv('/home/user/dolgov/hw/lenta_40k.csv.zip')

In [6]:
data.head(5)

,text,topic
0,Россия должна сотрудничать с Всемирным антидоп...,Спорт
1,Уголовный суд Кувейта 28 июня освободил под за...,Мир
2,Французский журнал Charlie Hebdo опубликовал н...,Интернет и СМИ
3,В Петербурге в доме № 53 по улице Лени Голиков...,Россия
4,"В московском аэропорту ""Домодедово"" задержан г...",Россия


In [7]:
data.topic.unique()

array(['Спорт', 'Мир', 'Интернет и СМИ', 'Россия', 'Наука и техника',
       'Силовые структуры', 'Бывший СССР', 'Культура', 'Экономика',
       'Ценности', 'Из жизни', 'Дом', 'Бизнес', '69-я параллель', 'Крым',
       'Путешествия', 'Культпросвет ', 'Легпром', 'Библиотека'],
      dtype=object)

In [8]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [9]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

len(vocab)

354611

In [11]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)


len(filtered_vocab)

24091

In [12]:
word2id = {'PAD':0, 'UNK':1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [13]:
X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [14]:
MAX_LEN = max(len(x) for x in X)

MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN, MEAN_LEN

(1748, 170.0)

In [15]:
# Увеличить размер эмбеддингов

MAX_LEN = int(MEAN_LEN + 50)

In [16]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)
X.shape

(44356, 220)

In [17]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [18]:
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])
len(label2id)

19

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

In [20]:
# 1) модель с 1 GRU слоем;

inputs_1 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_1 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_1, )

rnn_1 = tf.keras.layers.GRU(units=90, return_sequences=False)(embeddings_1)

outputs_1 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_1)

model_1 = tf.keras.Model(inputs=inputs_1, outputs=outputs_1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_1.summary()

2024-02-22 11:35:00.708198: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 11:35:00.774642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 11:35:00.775221: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding (Embedding)       (None, 220, 60)           1445580   
                                                                 
 gru (GRU)                   (None, 90)                41040     
                                                                 
 dense (Dense)               (None, 19)                1729      
                                                                 
Total params: 1488349 (5.68 MB)
Trainable params: 1488349 (5.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model_1.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20


2024-02-22 11:35:04.525351: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-22 11:35:05.181530: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2f8d212410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-22 11:35:05.181570: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-02-22 11:35:05.188714: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708590905.317583  552971 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 11s 186ms/step - loss: 2.5630 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3091 - val_f1: 0.0000e+00 - val_rec@prec: 4.5086e-04
Epoch 2/20
43/43 [==============================] - 7s 157ms/step - loss: 2.2408 - f1: 9.2653e-05 - rec@prec: 4.9836e-04 - val_loss: 2.1639 - val_f1: 0.0030 - val_rec@prec: 4.5086e-04
Epoch 3/20
43/43 [==============================] - 6s 149ms/step - loss: 2.0042 - f1: 0.0317 - rec@prec: 0.0060 - val_loss: 1.9169 - val_f1: 0.0932 - val_rec@prec: 0.0077
Epoch 4/20
43/43 [==============================] - 6s 138ms/step - loss: 1.7454 - f1: 0.1936 - rec@prec: 0.0662 - val_loss: 1.7952 - val_f1: 0.2225 - val_rec@prec: 0.0473
Epoch 5/20
43/43 [==============================] - 5s 105ms/step - loss: 1.5332 - f1: 0.3365 - rec@prec: 0.1730 - val_loss: 1.7237 - val_f1: 0.2626 - val_rec@prec: 0.0649
Epoch 6/20
43/43 [==============================] - 4s 104ms/step - loss: 1.3459 - f1: 0.4397 - rec@prec: 0.2874 - val_los

In [22]:
# 2) модель с 1 LSTM слоем

inputs_2 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_2 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_2, )

rnn_2 = tf.keras.layers.LSTM(units=90, return_sequences=False)(embeddings_2)

outputs_2 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_2)

model_2 = tf.keras.Model(inputs=inputs_2, outputs=outputs_2)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 lstm (LSTM)                 (None, 90)                54360     
                                                                 
 dense_1 (Dense)             (None, 19)                1729      
                                                                 
Total params: 1501669 (5.73 MB)
Trainable params: 1501669 (5.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model_2.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20
43/43 [==============================] - 7s 121ms/step - loss: 2.4958 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3182 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 98ms/step - loss: 2.3050 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.2725 - val_f1: 0.0000e+00 - val_rec@prec: 0.0027
Epoch 3/20
43/43 [==============================] - 4s 85ms/step - loss: 2.1936 - f1: 9.2930e-05 - rec@prec: 0.0023 - val_loss: 2.1290 - val_f1: 0.0000e+00 - val_rec@prec: 0.0014
Epoch 4/20
43/43 [==============================] - 4s 88ms/step - loss: 1.9471 - f1: 0.0753 - rec@prec: 0.0544 - val_loss: 1.8604 - val_f1: 0.2195 - val_rec@prec: 0.1060
Epoch 5/20
43/43 [==============================] - 4s 92ms/step - loss: 1.5570 - f1: 0.3773 - rec@prec: 0.2025 - val_loss: 1.6277 - val_f1: 0.4365 - val_rec@prec: 0.1438
Epoch 6/20
43/43 [==============================] - 3s 80ms/step - loss: 1.3290 - f1: 0.5692 - rec@prec: 0.3

In [24]:
# 3) модель с 1 GRU и 1 LSTM слоем

inputs_3 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_3 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_3, )

rnn_3_gru = tf.keras.layers.GRU(units=90, return_sequences=True)(embeddings_3)

rnn_3_lstm = tf.keras.layers.LSTM(units=90, return_sequences=False)(rnn_3_gru)

outputs_3 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_3_lstm)

model_3 = tf.keras.Model(inputs=inputs_3, outputs=outputs_3)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_3.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 gru_1 (GRU)                 (None, 220, 90)           41040     
                                                                 
 lstm_1 (LSTM)               (None, 90)                65160     
                                                                 
 dense_2 (Dense)             (None, 19)                1729      
                                                                 
Total params: 1553509 (5.93 MB)
Trainable params: 1553509 (5.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model_3.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20
43/43 [==============================] - 10s 163ms/step - loss: 2.4838 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3158 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 6s 129ms/step - loss: 2.1299 - f1: 0.0017 - rec@prec: 0.0000e+00 - val_loss: 1.8918 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 5s 111ms/step - loss: 1.7478 - f1: 0.0965 - rec@prec: 0.0433 - val_loss: 1.7020 - val_f1: 0.2895 - val_rec@prec: 0.0920
Epoch 4/20
43/43 [==============================] - 5s 128ms/step - loss: 1.5765 - f1: 0.2820 - rec@prec: 0.1123 - val_loss: 1.6437 - val_f1: 0.3376 - val_rec@prec: 0.1195
Epoch 5/20
43/43 [==============================] - 5s 114ms/step - loss: 1.4253 - f1: 0.3512 - rec@prec: 0.1732 - val_loss: 1.5355 - val_f1: 0.3818 - val_rec@prec: 0.1637
Epoch 6/20
43/43 [==============================] - 5s 118ms/step - loss: 1.2795 - f1: 0.4690 - rec@prec: 0.231

In [26]:
# 4) модель с 1 BIGRU и 2 LSTM слоями

inputs_4 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_4 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_4, )

rnn_4_bigru = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(units=90, return_sequences=True)
    )(embeddings_4)

rnn_4_lstm_1 = tf.keras.layers.LSTM(units=90, return_sequences=True)(rnn_4_bigru)

rnn_4_lstm_2 = tf.keras.layers.LSTM(units=90, return_sequences=False)(rnn_4_lstm_1)

outputs_4 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_4_lstm_2)

model_4 = tf.keras.Model(inputs=inputs_4, outputs=outputs_4)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_4.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_4.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 bidirectional (Bidirection  (None, 220, 180)          82080     
 al)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 220, 90)           97560     
                                                                 
 lstm_3 (LSTM)               (None, 90)                65160     
                                                                 
 dense_3 (Dense)             (None, 19)                1729      
                                                           

In [27]:
model_4.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20
43/43 [==============================] - 15s 228ms/step - loss: 2.4445 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3208 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 8s 189ms/step - loss: 2.2984 - f1: 0.0000e+00 - rec@prec: 1.8985e-04 - val_loss: 2.2640 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 7s 174ms/step - loss: 1.9811 - f1: 0.0518 - rec@prec: 0.0254 - val_loss: 1.9110 - val_f1: 0.0746 - val_rec@prec: 0.0397
Epoch 4/20
43/43 [==============================] - 7s 167ms/step - loss: 1.7366 - f1: 0.1489 - rec@prec: 0.0854 - val_loss: 1.7238 - val_f1: 0.1941 - val_rec@prec: 0.1159
Epoch 5/20
43/43 [==============================] - 7s 169ms/step - loss: 1.6179 - f1: 0.2507 - rec@prec: 0.1200 - val_loss: 1.6620 - val_f1: 0.2397 - val_rec@prec: 0.1402
Epoch 6/20
43/43 [==============================] - 7s 162ms/step - loss: 1.4602 - f1: 0.3398 - rec@prec: 0

In [28]:
# 5) модель с 5 GRU слоями и 3 LSTM слоями

inputs_5 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_5 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_5, )

rnn_5_gru_1 = tf.keras.layers.GRU(units=90, return_sequences=True)(embeddings_5)

rnn_5_gru_2 = tf.keras.layers.GRU(units=90, return_sequences=True)(rnn_5_gru_1)

rnn_5_lstm_1 = tf.keras.layers.LSTM(units=90, return_sequences=True)(rnn_5_gru_2)

rnn_5_gru_3 = tf.keras.layers.GRU(units=90, return_sequences=True)(rnn_5_lstm_1)

rnn_5_lstm_2 = tf.keras.layers.LSTM(units=90, return_sequences=True)(rnn_5_gru_3)

rnn_5_gru_4 = tf.keras.layers.GRU(units=90, return_sequences=True)(rnn_5_lstm_2)

rnn_5_lstm_3 = tf.keras.layers.LSTM(units=90, return_sequences=True)(rnn_5_gru_4)

rnn_5_gru_5 = tf.keras.layers.GRU(units=90, return_sequences=False)(rnn_5_lstm_3)

outputs_5 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_5_gru_5)

model_5 = tf.keras.Model(inputs=inputs_5, outputs=outputs_5)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_5.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_5.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 gru_3 (GRU)                 (None, 220, 90)           41040     
                                                                 
 gru_4 (GRU)                 (None, 220, 90)           49140     
                                                                 
 lstm_4 (LSTM)               (None, 220, 90)           65160     
                                                                 
 gru_5 (GRU)                 (None, 220, 90)           49140     
                                                                 
 lstm_5 (LSTM)               (None, 220, 90)           6516

In [29]:
model_5.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20


43/43 [==============================] - 26s 338ms/step - loss: 2.4224 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3231 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 12s 271ms/step - loss: 2.3237 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3211 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 11s 261ms/step - loss: 2.3249 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3212 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 11s 252ms/step - loss: 2.3232 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3200 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 5/20
43/43 [==============================] - 11s 262ms/step - loss: 2.3240 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3205 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 6/20
43/43 [==============================] - 11s 248ms/step

In [30]:
# 6) модель 1 BIGRU и 1 BILSTM слоями, причем так чтобы модели для forward и backward прохода отличалис

inputs_6 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_6 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_6, )

forward_1 = tf.keras.layers.GRU(units=90, return_sequences=True)
backward_1 = tf.keras.layers.GRU(units=80, return_sequences=True, go_backwards=True)
rnn_6_bigru = tf.keras.layers.Bidirectional(forward_1, backward_layer=backward_1)(embeddings_6)

forward_2 = tf.keras.layers.LSTM(units=80, return_sequences=True)
backward_2 = tf.keras.layers.LSTM(units=70, return_sequences=True, go_backwards=True)
rnn_6_bilstm = tf.keras.layers.Bidirectional(forward_2, backward_layer=backward_2)(rnn_6_bigru)

rnn_6_gru = tf.keras.layers.GRU(units=70, return_sequences=False)(rnn_6_bilstm)

outputs_6 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_6_gru)

model_6 = tf.keras.Model(inputs=inputs_6, outputs=outputs_6)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_6.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_6.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 bidirectional_1 (Bidirecti  (None, 220, 170)          75120     
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 220, 150)          147800    
 onal)                                                           
                                                                 
 gru_10 (GRU)                (None, 70)                46620     
                                                                 
 dense_5 (Dense)             (None, 19)                1349

In [31]:
model_6.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20
43/43 [==============================] - 19s 239ms/step - loss: 2.4574 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3006 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 8s 194ms/step - loss: 2.0783 - f1: 0.0256 - rec@prec: 0.0039 - val_loss: 1.8388 - val_f1: 0.1210 - val_rec@prec: 0.0595
Epoch 3/20
43/43 [==============================] - 8s 192ms/step - loss: 1.6253 - f1: 0.2372 - rec@prec: 0.1254 - val_loss: 1.5292 - val_f1: 0.2898 - val_rec@prec: 0.1713
Epoch 4/20
43/43 [==============================] - 8s 177ms/step - loss: 1.3188 - f1: 0.4233 - rec@prec: 0.2679 - val_loss: 1.3510 - val_f1: 0.5058 - val_rec@prec: 0.2998
Epoch 5/20
43/43 [==============================] - 7s 170ms/step - loss: 1.0053 - f1: 0.6699 - rec@prec: 0.5826 - val_loss: 1.1909 - val_f1: 0.6505 - val_rec@prec: 0.5122
Epoch 6/20
43/43 [==============================] - 7s 166ms/step - loss: 0.7569 - f1: 0.7798 - rec@prec: 0.7726 - val_loss

In [32]:
# 7) модель, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM

inputs_7 = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings_7 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs_7, )

rnn_7_lstm_1 = tf.keras.layers.LSTM(units=90, return_sequences=True)(embeddings_7)

rnn_7_gru_1 = tf.keras.layers.GRU(units=90, return_sequences=True)(rnn_7_lstm_1)

forward_1 = tf.keras.layers.LSTM(units=90, return_sequences=True)
backward_1 = tf.keras.layers.LSTM(units=80, return_sequences=True, go_backwards=True)
rnn_7_bilstm = tf.keras.layers.Bidirectional(forward_1, backward_layer=backward_1)(rnn_7_gru_1)

forward_2 = tf.keras.layers.GRU(units=80, return_sequences=True)
backward_2 = tf.keras.layers.GRU(units=70, return_sequences=True, go_backwards=True)
rnn_7_bigru = tf.keras.layers.Bidirectional(forward_2, backward_layer=backward_2)(rnn_7_bilstm)

rnn_7_gru_2 = tf.keras.layers.GRU(units=90, return_sequences=True)(rnn_7_bigru)

rnn_7_lstm_2 = tf.keras.layers.LSTM(units=90, return_sequences=False)(rnn_7_gru_2)

outputs_7 = tf.keras.layers.Dense(len(label2id), activation='softmax')(rnn_7_lstm_2)

model_7 = tf.keras.Model(inputs=inputs_7, outputs=outputs_7)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_7.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1,
                       tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model_7.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 220)]             0         
                                                                 
 embedding_6 (Embedding)     (None, 220, 60)           1445580   
                                                                 
 lstm_9 (LSTM)               (None, 220, 90)           54360     
                                                                 
 gru_11 (GRU)                (None, 220, 90)           49140     
                                                                 
 bidirectional_3 (Bidirecti  (None, 220, 170)          119880    
 onal)                                                           
                                                                 
 bidirectional_4 (Bidirecti  (None, 220, 150)          111300    
 onal)                                                     

In [33]:
model_7.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=20)

Epoch 1/20
43/43 [==============================] - 25s 329ms/step - loss: 2.4272 - f1: 0.0000e+00 - rec@prec: 0.0000e+00 - val_loss: 2.3219 - val_f1: 0.0000e+00 - val_rec@prec: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 11s 262ms/step - loss: 2.2341 - f1: 0.0010 - rec@prec: 0.0047 - val_loss: 2.1146 - val_f1: 0.0666 - val_rec@prec: 0.0464
Epoch 3/20
43/43 [==============================] - 11s 267ms/step - loss: 1.8215 - f1: 0.1723 - rec@prec: 0.0782 - val_loss: 1.7734 - val_f1: 0.1786 - val_rec@prec: 0.0983
Epoch 4/20
43/43 [==============================] - 10s 244ms/step - loss: 1.6053 - f1: 0.3478 - rec@prec: 0.1522 - val_loss: 1.6949 - val_f1: 0.3595 - val_rec@prec: 0.1637
Epoch 5/20
43/43 [==============================] - 11s 249ms/step - loss: 1.4866 - f1: 0.4272 - rec@prec: 0.2520 - val_loss: 1.5885 - val_f1: 0.4214 - val_rec@prec: 0.2480
Epoch 6/20
43/43 [==============================] - 10s 243ms/step - loss: 1.3078 - f1: 0.5187 - rec@prec: 0.3528 - val

In [34]:
models = [
    model_1,
    model_2,
    model_3,
    model_4,
    model_5,
    model_6,
    model_7
]

In [35]:
f1s = []
for i, model in enumerate(models):
    f1 = (model.get_metrics_result()['f1']).numpy()
    f1s.append((i, f1))
    print(f"Модель №{i+1}, F1 - {f1}")

maxf1 = max(f1s)
print(f"С F1 {maxf1[1]} лучшая модель - №{maxf1[0]}")

Модель №1, F1 - 0.6271504759788513
Модель №2, F1 - 0.6899943351745605
Модель №3, F1 - 0.6119385361671448
Модель №4, F1 - 0.6414610743522644
Модель №5, F1 - 0.6066408157348633
Модель №6, F1 - 0.6752133369445801
Модель №7, F1 - 0.6411024928092957
С F1 0.6411024928092957 лучшая модель - №6


## Задание 2 (6 баллов)


На данных википедии (wikiann) обучите 2 модели:  
1) модель в которой будут использованы предобученные эмбединги слов и несколько BILSTM слоев.
1) модель в которой будут использованы предобученные эмбединги слов и несколько BIGRU слоев.

Сравните качество по метрикам. Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

In [38]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 50.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0


In [39]:
from datasets import load_dataset

/home/user/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
dataset = load_dataset("wikiann", 'ru')
dataset

FileNotFoundError: Unable to find 'hf://datasets/wikiann@dc60b43e8e7e1c939a18bf5da8aac1b3f174cab6/ru/validation/0000.parquet' with any supported extension ['.csv', '.tsv', '.json', '.jsonl', '.parquet', '.geoparquet', '.gpq', '.arrow', '.txt', '.tar', '.blp', '.bmp', '.dib', '.bufr', '.cur', '.pcx', '.dcx', '.dds', '.ps', '.eps', '.fit', '.fits', '.fli', '.flc', '.ftc', '.ftu', '.gbr', '.gif', '.grib', '.h5', '.hdf', '.png', '.apng', '.jp2', '.j2k', '.jpc', '.jpf', '.jpx', '.j2c', '.icns', '.ico', '.im', '.iim', '.tif', '.tiff', '.jfif', '.jpe', '.jpg', '.jpeg', '.mpg', '.mpeg', '.msp', '.pcd', '.pxr', '.pbm', '.pgm', '.ppm', '.pnm', '.psd', '.bw', '.rgb', '.rgba', '.sgi', '.ras', '.tga', '.icb', '.vda', '.vst', '.webp', '.wmf', '.emf', '.xbm', '.xpm', '.BLP', '.BMP', '.DIB', '.BUFR', '.CUR', '.PCX', '.DCX', '.DDS', '.PS', '.EPS', '.FIT', '.FITS', '.FLI', '.FLC', '.FTC', '.FTU', '.GBR', '.GIF', '.GRIB', '.H5', '.HDF', '.PNG', '.APNG', '.JP2', '.J2K', '.JPC', '.JPF', '.JPX', '.J2C', '.I

In [ ]:
vocab = Counter()

for text in data.tokens:
    vocab.update(preprocess(text))

len(vocab)

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)


len(filtered_vocab)

In [ ]:
word2id = {'PAD':0, 'UNK':1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [ ]:
X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [ ]:
MAX_LEN = max(len(x) for x in X)

MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN, MEAN_LEN

MAX_LEN = int(MEAN_LEN + 50)

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)
X.shape

In [ ]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [ ]:
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])
len(label2id)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)